# SCP Toolbox Workshop

___A tutorial on generating dynamically feasible trajectories reliably and efficiently___

Monday, February 7, 2022

Rocky Mountain AAS GN&C Conference, Breckenridge, CO

In [1]:
import Pkg
Pkg.activate("..")

# these lines are required only for local installations
Pkg.develop(path="../../scp_traj_opt/")
Pkg.precompile()

Pkg.add("JuMP")
Pkg.build("JuMP")

using SCPToolbox
using JuMP
using ECOS
using SCS
using Ipopt
using Printf

  Activating project at `~/GitHub/SCPToolbox_tutorial`
   Resolving package versions...
  No Changes to `~/GitHub/SCPToolbox_tutorial/Project.toml`
  No Changes to `~/GitHub/SCPToolbox_tutorial/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/GitHub/SCPToolbox_tutorial/Project.toml`
  No Changes to `~/GitHub/SCPToolbox_tutorial/Manifest.toml`


# Part 2: Simple Conic Linear Program Example

Solve the following nonconvex optimization problem as a simple conic linear program subproblem:

\begin{align}
\underset{z}{\text{minimize}} &\quad L(z) \triangleq z_2 \\
\text{subject to} &\quad \color{#000080}{z_2 - z_1^2 = 0}\\
 &\quad \color{#008000}{z_2 + 0.1 z_1 = 0.6}\\
 &\quad \color{#FF00FF}{z_1 \leq 0.85}
\end{align}

<center>
    <br />
    <img src="media/crawling.png" width="450"/>
    <br />
    <b>Figure.</b>  A two-dimensional nonconvex toy problem that exemplifies a convex subproblem obtained during an SCP iteration.
    <br />
</center>

In this case, the cost function $L(z) = z_2$ and level curves of the cost are shown as gray dashed lines. The blue curve represents a nonconvex equality constraint, and its linearization is shown as the blue dash-dot line. Another convex equality constraint is shown in green, and a convex inequality constraint is shown as the vertical red dashed line. The trust region is the red circle centered at the linearization point $\bar{z}$, and has radius $\eta$. The optimal solution of the original (non-approximated) problem is shown as the black square. The convex subproblem is artificially infeasible. Without the trust region and the green constraint, it would also be artifically unbounded.  

To convert this problem into a CLP subproblem, we first generate (convex, linear) hyperplane
approximation of the nonconvex constraint. We also include virtual buffers $\nu$ to avoid artificial infeasibility, and a trust region upper bound $\eta$ to avoid artificial unboundedness.

Penalties for these terms are augmented to the cost function. Virtual control magnitude is penalized to incentivize convergence to zero once feasibility of the subproblem is obtained. In fact, the virtual control penalty has a higher weight in the augmented objective. Trust region radius is penalized to incentivize the optimal solution of the subproblem at the current iteration to stay close to the linearization point, where the approximation of the nonconvex constraint is valid. Furthermore, the trust region constraint precludes artificial unboundedness that may arise from the feasible set becoming unbounded due to the linearized constraint.

\begin{align}
\underset{z}{\text{minimize}} &\quad z_2 + w_{\text{tr}}\color{#FF0000}{\eta} + w_{\text{vb}}\color{#000080}{\|\nu\|_2} \\
\text{subject to} &\quad  \color{#000080}{(-2\bar{z}_1)z_1 + z_2 + \bar{z}_1^2 + \nu = 0}  \\
 &\quad \color{#008000}{z_2 + 0.1 z_1 = 0.6}\\
 &\quad \color{#FF00FF}{z_1 \leq 0.85} \\
 &\quad \color{#FF0000 }{\| z - \bar{z} \|_2 \leq \eta}
\end{align}

The subproblem described above is still not a CLP since the objective function is not linear. The virtual buffer penalty is converted to a conic constraint via the epigraph form by introducing a slack variable $\mu$.

\begin{align}
\underset{z}{\text{minimize}} &\quad z_2 + w_{\text{tr}}\color{#FF0000}{\eta} + w_{\text{vb}}\color{#000080}{\mu}\\
\text{subject to} &\quad  \color{#000080}{(-2\bar{z}_1)z_1 + z_2 + \bar{z}_1^2 + \nu = 0}  \\
 &\quad \color{#008000}{z_2 + 0.1 z_1 = 0.6}\\
 &\quad \color{#FF00FF}{z_1 \leq 0.85} \\
 &\quad \color{#FF0000 }{\| z - \bar{z} \|_2 \leq \eta} \\
 &\quad \color{#000080}{\|\nu\|_2 \leq \mu}
\end{align}

## Constraints

In [2]:
# Linearized nonconvex constraint
lin_ncvx(z, ν, pbm) = @add_constraint(
        pbm, ZERO, "my-ncvx", (z,ν) -> cst["A1"]*z .+ cst["b1"] .+ ν);

In [3]:
# Hyperplane constraint
hypln(z, pbm) = @add_constraint(
     pbm, ZERO, "my-hypln", (z,) -> cst["A2"]*z .+ cst["b2"]);

In [4]:
# Halfspace constraint
hlfsp(z, pbm) = @add_constraint(
      pbm, NONPOS, "my-hlfspace", (z,) -> cst["A3"]*z .+ cst["b3"]);

In [5]:
# Trust-region penalty constraint
tr_cnstr(z, η, pbm) = @add_constraint(
         pbm, SOC, "my-soc", (z,η) -> vcat(η,z .- cst["z̄"]));

In [6]:
# Virtual buffer penalty constraint
vb_cnstr(ν, μ, pbm) = @add_constraint(
         pbm, SOC, "my-soc2", (ν,μ) -> vcat(μ,ν));

## Cost function

In [7]:
cost_fun(η, μ, z, pbm) = @add_cost(
         pbm, (η,μ,z) -> z[2] .+ cst["wtr"]*η .+ cst["wvb"]*μ);

In [8]:
# ..:: Construct Subproblem ::..
function construct_subpbm(params)
    
    # Construct the subproblem
    pbm = ConicProgram(params;solver = params["solver"],solver_options = params["opts"])
    
    # Create variables
    z = @new_variable(pbm, 2, "z")
    ν = @new_variable(pbm, "ν")
    η = @new_variable(pbm, "η")
    μ = @new_variable(pbm, "μ")  
        
    # Linearized nonconvex constraint
    lin_ncvx(z, ν, pbm)
    
    # Hyperplane constraint
    hypln(z, pbm)
    
    # Halfspace constraint
    hlfsp(z, pbm)
        
    # Trust-region penalty constraint
    tr_cnstr(z, η, pbm)
    
    # Virtual buffer penalty constraint
    vb_cnstr(ν, μ, pbm)
    
    # Cost function
    cost_fun(η, μ, z, pbm)
    
    return pbm, z, η, μ
    
end;

## SCP configuration

In [9]:
# Construct problem parameters
params = Dict()
params["solver"] = ECOS
params["opts"]   = Dict("verbose"=>0,"abstol"=>1e-8)
# params["opts"] = Dict("print_level"=>0)
# params["opts"] = Dict("verbose"=>0)

# Initial reference and constraint data
params["z̄"] = zeros(2);
params["A1"] = Matrix(transpose([-2*params["z̄"][1];1.0]))
params["b1"] = params["z̄"][1]^2
params["A2"] = Matrix(transpose([0.1;1]))
params["b2"] = -0.6
params["A3"] = Matrix(transpose([1.0;0.0]))
params["b3"] = -0.85

iter_max = 30; iter_conv = copy(iter_max)
ϵ_conv = 1e-8
params["wtr"] = 10.0
params["wvb"] = 10000.0
z_hist = [zeros(2) for k = 1:iter_max]
μ_hist = zeros(iter_max)
η_hist = zeros(iter_max)
cost_hist = zeros(iter_max);

## SCP solve step

In [10]:
# Iteration counter
k = 1
while true
    
    # Construct subproblem
    pbm, z, η, μ = construct_subpbm(params)
    
    # Solve subproblem
    exit_flag = solve!(pbm)
   
    # Save history of the subproblem
    z_hist[k] .= value(z)    
    η_hist[k] = value(η)[1]
    μ_hist[k] = value(μ)[1]
    cost_hist[k] = z_hist[k][2] 

    # Check stopping criterion
    bool_conv = max(η_hist[k],μ_hist[k]) <= ϵ_conv
    
    @printf("Iteration %d | %s | Trust region: %7.1e | Virtual buffer: %7.1e | Cost: %.2f |\n",k,string(exit_flag),η_hist[k],abs(μ_hist[k]),cost_hist[k])
        
    # Stop at maximum iterations
    if (k == iter_max) || bool_conv
        iter_conv = k        
        break
    else
    
        # Update next reference trajectory as solution
        params["z̄"] = z_hist[k]

        # Create linearized constraint values
        params["A1"] = Matrix(transpose([-2*params["z̄"][1];1.0]))
        params["b1"] = params["z̄"][1]^2    
        
        k += 1
    end
end


Iteration 1 | OPTIMAL | Trust region: 9.9e-01 | Virtual buffer: 5.1e-01 | Cost: 0.51 |
Iteration 2 | OPTIMAL | Trust region: 1.2e-01 | Virtual buffer: 6.2e-14 | Cost: 0.53 |
Iteration 3 | OPTIMAL | Trust region: 8.5e-03 | Virtual buffer: 1.9e-14 | Cost: 0.53 |
Iteration 4 | OPTIMAL | Trust region: 4.6e-05 | Virtual buffer: 1.0e-12 | Cost: 0.53 |
Iteration 5 | OPTIMAL | Trust region: 6.5e-13 | Virtual buffer: 2.4e-14 | Cost: 0.53 |


In [11]:
print("\n\nOptimal solution: $(z_hist[iter_conv])");



Optimal solution: [0.7262087349048842, 0.5273791267502363]

# Validating SCP solution via JuMP

In [12]:
# define JuMP model and NLP solver
model = Model(Ipopt.Optimizer)
set_silent(model)

# declare variables
@variable(model,z[1:2])

# nonconvex quadratic equality constraint
@NLconstraint(model,z[2]-z[1]^2 == 0)

# convex constraints
@constraint(model,params["A2"]*z .+ params["b2"] .== 0)
@constraint(model,params["A3"]*z .+ params["b3"] .≤ 0)
@objective(model,Min,z[2])

# initial guess to NLP solver
set_start_value.(all_variables(model),ones(2))

# parse and call to solver
optimize!(model);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [13]:
print("\n\nOptimal solution: $(JuMP.value.(z))");



Optimal solution: [0.7262087348131022, 0.5273791265186897]